In [0]:
# File path for the Excel file and DBFS storage
source_blob_path = "abfss://blobstorage@datablobstorage001.dfs.core.windows.net/landingzone/Excel"
local_file_path = "/dbfs/dbfs/tmp/current_excel_file.xlsx"  # Local temp file in DBFS
local_file_path_1 = "dbfs:/dbfs/tmp/current_excel_file.xlsx"
parquet_folder_path = "/dbfs/tmp/parquet_files"  # Parquet file storage location on DBFS

In [0]:
files = dbutils.fs.ls(source_blob_path)
excel_files = [f.path for f in files if f.path.endswith(".xlsx")]

if not excel_files:
    raise FileNotFoundError(f"No .xlsx files found in {blob_path}")
    
# Use the first Excel file detected
excel_file_remote = excel_files[0]
dbutils.fs.cp(excel_file_remote, local_file_path)
print(f"Copied file {excel_file_remote} to {local_file_path}")
files1 = dbutils.fs.ls("/dbfs/tmp/current_excel_file.xlsx")
print("File Excel in DBFS", files1)

In [0]:
import shutil
from openpyxl import load_workbook
import os

# Copy from DBFS to local file system
dbfs_path = local_file_path
local_path = "/tmp/current_excel_file.xlsx"

if not os.path.exists(dbfs_path):
    raise FileNotFoundError(
        f"File not found at {dbfs_path}. Please upload the file to DBFS using the Databricks UI or dbutils.fs.cp."
    )
    
shutil.copy(dbfs_path, local_path)

try:
    workbook = load_workbook(local_path, read_only=True)
    sheet_names = workbook.sheetnames
    print(sheet_names)
except Exception as e:
    raise RuntimeError(f"Failed to fetch sheet names: {e}")


In [0]:
# dbutils.fs.mkdirs("/dbfs/dbfs/parquet_files")
# dbutils.fs.rm("dbfs:/dbfs/tmp/parquet_files", recurse=True)
# dbutils.fs.rm("dbfs:/dbfs/dbfs/parquet_files")

In [0]:
import re

# List of sheet names from the Excel file
print(sheet_names)

# Function to clean sheet names
def clean_sheet_name(sheet_name):
    """
    Cleans a sheet name to make it suitable for use as a parquet file name.
    Replaces invalid characters with underscores, removes trailing/leading spaces, 
    and ensures lowercase for case-insensitivity.
    """
    sanitized_name = re.sub(r"[^\w]", "_", sheet_name.strip())  # Replace non-alphanumeric characters with "_"
    sanitized_name = re.sub(r"__+", "_", sanitized_name)  # Replace multiple underscores with a single "_"
    return sanitized_name.lower()  # Convert to lowercase

def does_parquet_file_exist(sheet_name, base_dir):
    """
    Checks if a Parquet file for the sheet exists in the specified directory.
    :param sheet_name: Name of the sheet.
    :param base_dir: Directory where Parquet files are stored.
    :return: True if the Parquet file exists, False otherwise.
    """
    parquet_file_path = f"{base_dir}/{sheet_name}.parquet"
    
    try:
        # Attempt to list the file's parent directory
        parent_dir = parquet_file_path.rsplit("/", 1)[0]
        files = dbutils.fs.ls(parent_dir)
        print("files: ", files)
        # Check if the exact file exists
        return any(file.path == parquet_file_path or file.path.rstrip("/") == parquet_file_path for file in files)
    except Exception:
        # Directory or file doesn't exist
        return False

cleaned_sheet_names = [clean_sheet_name(sheet_name) for sheet_name in sheet_names]
# Print the original and cleaned sheet names
print("Original and Cleaned Sheet Names:")
for original, cleaned in zip(sheet_names, cleaned_sheet_names):
    print(f"Original: {original} --> Cleaned: {cleaned}")

# Verify if the Parquet file exists for each sheet
parquet_file_path = "/dbfs/dbfs/parquet_files"
dir_parquet_files = {}
for sheet_name in cleaned_sheet_names:
    if does_parquet_file_exist(sheet_name, parquet_file_path):
        print(f"Parquet file for sheet '{sheet_name}' already exists.")
        dir_parquet_files[sheet_namer] = True
    else:
        print(f"Parquet file for sheet '{sheet_name}' does not exist.")
        dir_parquet_files[sheet_name] = False

print(dir_parquet_files)

In [0]:
import re
import os
from pyspark.sql.functions import when, lit, col, regexp_replace
from pyspark.sql.types import DecimalType, LongType


# Step 1: Fully clean and map original column names to cleaned names
def map_and_clean_column_names(columns):
    """
    Cleans and maps column names:
    - Strips leading/trailing whitespace and newlines.
    - Replaces invalid characters (e.g., spaces, symbols) with underscores.
    - Converts names to lowercase.
    Returns:
    - A list of cleaned column names.
    - A mapping dictionary (original -> cleaned).
    """
    original_to_cleaned = {
        col_name.strip(): re.sub(r"[^\w]", "_", col_name.strip()).lower()
        for col_name in columns
    }
    cleaned_columns = list(original_to_cleaned.values())
    return cleaned_columns, original_to_cleaned


# Step 2: Dynamically align and reconcile schema
def align_schema_dynamically(df, parquet_path, original_to_cleaned):
    """
    Aligns the schema of the DataFrame dynamically:
    - Adds missing columns as nulls.
    - Aligns columns with the Parquet schema (if it exists).
    - Ensures column order consistency.
    """
    try:
        # Load existing Parquet schema if the file exists
        if os.path.exists(parquet_path):
            existing_parquet_df = spark.read.parquet(parquet_path)
            existing_schema = {
                col_name.strip(): dtype for col_name, dtype in existing_parquet_df.dtypes
            }
            print(f"[Info] Successfully loaded existing schema from {parquet_path}")
        else:
            raise FileNotFoundError(f"Parquet file not found: {parquet_path}")
    except Exception as e:
        # Fallback to the current DataFrame schema if no Parquet file exists
        print(f"[Warning] No Parquet schema found: {e}. Using DataFrame schema as baseline.")
        existing_schema = {col_name.strip(): dtype for col_name, dtype in df.dtypes}

    # Reverse mapping: Cleaned to Original Names
    cleaned_to_original = {v: k for k, v in original_to_cleaned.items()}

    # Add missing columns dynamically
    for col_name, col_dtype in existing_schema.items():
        cleaned_col_name = original_to_cleaned.get(col_name.strip(), col_name)  # Handle cleaned names
        if cleaned_col_name not in df.columns:
            print(f"[Adding Missing Column] Original: {col_name}, Cleaned: {cleaned_col_name}")
            df = df.withColumn(cleaned_col_name, lit(None).cast(col_dtype))  # Add missing column

    # Reorder DataFrame columns to match Parquet schema
    ordered_columns = [original_to_cleaned.get(col, col) for col in existing_schema.keys()]
    df = df.select(*ordered_columns)
    print(f"[Schema Alignment Debug] Aligned Columns: {ordered_columns}")
    return df


# Step 3: Normalize numeric columns
def normalize_numeric_columns(df):
    """
    Normalizes numeric columns stored as strings by applying these rules:
    - If the column contains letters, leave it as StringType.
    - If the column is numeric with commas or decimals, convert it to DecimalType.
    - If the column is numeric without commas or decimals, convert it to LongType.
    """
    for col_name, dtype in df.dtypes:
        # Skip non-string columns, as there's no normalization required for them
        if dtype != "string":
            continue

        print(f"[Normalizing Numeric Column] {col_name}")

        # Clean and remove invalid characters like spaces and commas for processing
        cleaned_col = regexp_replace(col(col_name), "[^\d.,a-zA-Z]", "")

        # Apply rules:
        df = df.withColumn(
            col_name,
            when(cleaned_col.rlike(r"[a-zA-Z]+"), col(col_name))  # Leave as-is if it contains letters
            .when(cleaned_col.rlike(r"^\d+,\d+$|^\d+\.\d+$"), cleaned_col.cast(DecimalType(38, 10)))  # Decimal case
            .when(cleaned_col.rlike(r"^\d+$"), cleaned_col.cast(LongType()))  # Integer case
            .otherwise(col(col_name))  # Retain original content if none of the rules match
        )

    return df

# Step 4: Process all sheets from the Excel file
def process_excel_sheets(sheet_names, excel_file_path, parquet_dir):
    """
    Processes all sheets in an Excel file:
    - Cleans and normalizes schema.
    - Dynamically aligns and writes to Parquet.
    """
    for sheet_name in sheet_names:
        try:
            print(f"\n[Processing Sheet] {sheet_name}")

            # Load sheet as DataFrame
            df = spark.read.format("com.crealytics.spark.excel") \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .option("dataAddress", f"'{sheet_name}'!A1") \
                .load(excel_file_path)

            print(f"[Original Columns Debug] {sheet_name}: {df.columns}")

            # Step 1: Clean column names and create mapping
            cleaned_columns, original_to_cleaned = map_and_clean_column_names(df.columns)

            # Rename all columns in DataFrame according to the mapping
            for original_col, cleaned_col in original_to_cleaned.items():
                if original_col != cleaned_col:
                    df = df.withColumnRenamed(original_col, cleaned_col)

            print(f"[Cleaned Columns Debug] {sheet_name}: {df.columns}")

            # Step 2: Define Parquet path
            sanitized_sheet_name = re.sub(r"[^\w]", "_", sheet_name.strip().lower())
            parquet_path = os.path.join(parquet_dir, f"{sanitized_sheet_name}.parquet")

            # Step 3: Align schema dynamically
            df = align_schema_dynamically(df, parquet_path, original_to_cleaned)

            # Step 4: Normalize numeric columns
            df = normalize_numeric_columns(df)

            # Step 5: Debug schema before writing
            print(f"[Final Schema Debug] {sheet_name}: {df.schema.simpleString()}")

            # Step 6: Write aligned and cleaned DataFrame to Parquet
            df.write.mode("overwrite").parquet(parquet_path)
            print(f"[Success] Sheet '{sheet_name}' written to Parquet at: {parquet_path}")

        except Exception as e:
            print(f"[Error] Failed to process sheet: {sheet_name}. Error: {e}")


# Example Execution
local_excel_path = "/dbfs/tmp/current_excel_file.xlsx"
parquet_dir = "/dbfs/tmp/parquet_files"

process_excel_sheets(sheet_names, local_excel_path, parquet_dir)

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW
bushing_parquet
USING parquet
OPTIONS (path "/dbfs/tmp/parquet_files/bushings.parquet");

SELECT *
FROM bushing_parquet
where part_number IS NOT NULL;

In [0]:
file_path = "/dbfs/tmp/parquet_files/bushings.parquet"
df = spark.read.parquet(file_path)
display(df)